In [2]:
import torch
import numpy as np

In [3]:
def create_toy_data(func, sample_size, std, domain=[0, 1]):
    x = np.linspace(domain[0], domain[1], sample_size)
    t = func(x) + np.random.normal(scale=std, size=x.shape)
    return x, t

def cubic(x):
    return x * (x - 5) * (x + 5)

x, y = create_toy_data(cubic, 100, 10., [-5, 5])



In [4]:
X = torch.from_numpy(x).float()
X = torch.stack([torch.ones(100), X, X**2, X**3]).T
y = torch.from_numpy(y).float()

In [5]:
class VariationalLinearRegression:
    def __init__(self, beta=1, a0=1, b0=1, iters=100):
        self.beta = beta
        self.a0 = a0
        self.b0 = b0
        self.iters = iters
    
    def fit(self, X, y):
        XtX = torch.matmul(X.T, X)
        M = X.shape[1]
        self.a = self.a0 + M / 2
        self.b = self.b0
        for _ in range(self.iters):
            prev_b = self.b
            w_prec = self.beta * XtX
            w_prec[np.diag_indices(M)] += self.a / self.b
            self.w_var = w_prec.inverse()
            self.w_mean = self.beta * torch.matmul(self.w_var.mm(X.T), y)
            
            self.b = self.b0 + 0.5 * (self.w_mean ** 2).sum() + self.w_var.trace()
            
            if (prev_b - self.b).abs() < 1e-3:
                break
    
    def predict(self, X):
        y = torch.matmul(X, self.w_mean)
        y_var = 1 / self.beta + (torch.matmul(X, self.w_var) * X).sum(dim=1)
        y_std = torch.sqrt(y_var)
        return y, y_std

In [6]:
clf = VariationalLinearRegression(beta=0.01)

In [7]:
clf.fit(X, y)

In [8]:
pred, std = clf.predict(X)